<a href="https://colab.research.google.com/github/erica-mccormick/erica-mccormick.github.io/blob/main/get_tiffs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Generate and export tifs of CONUS at 500 m resolution with no mask**

## To ensure correct results, run each section from beginning to end. There are some repeated var names so be mindful!

##To plot flattened rasters based on lithological groups, see [here](https://github.com/erica-mccormick/rm_lithology/blob/main/lith.ipynb).

## **Imports**

In [1]:
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=1NaDIK5sB-6M_59EMsg_ADrF6G5xW99TmeFvrBuhqeo&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AY0e-g5JR3xqTdUcaaa_y3PeJSgwtGeP5yp4fhfmgBqfaTdlhsBRVnNjhUM

Successfully saved authorization token.


In [2]:
prism = ee.ImageCollection("OREGONSTATE/PRISM/AN81d")
pml = ee.ImageCollection("CAS/IGSNRR/PML/V2")
USGS_landcover = ee.ImageCollection("USGS/NLCD")
modis_landcover = ee.ImageCollection("MODIS/006/MCD12Q1")
snow_cover = ee.ImageCollection("MODIS/006/MOD10A1").select('NDSI_Snow_Cover')
land_cover = ee.ImageCollection("MODIS/006/MCD12Q1")

landfire = ee.ImageCollection("LANDFIRE/Vegetation/EVT/v1_4_0").first()

In [3]:
#Geometries
ca = ee.Feature(ee.FeatureCollection("users/daviddralle/ca_et/CA").first())
conus = ee.FeatureCollection('users/ericamccormick/20_RockMoisture/geometries/conus_20m')
texas = ee.FeatureCollection('users/ericamccormick/20_RockMoisture/geometries/TX')

#Dbedrock,Y
drock04 = ee.Image('users/ericamccormick/20_RockMoisture/deficit/bedrock/Dbedrock_2004')
drock05 = ee.Image('users/ericamccormick/20_RockMoisture/deficit/bedrock/Dbedrock_2005')
drock06 = ee.Image('users/ericamccormick/20_RockMoisture/deficit/bedrock/Dbedrock_2006')
drock07 = ee.Image('users/ericamccormick/20_RockMoisture/deficit/bedrock/Dbedrock_2007')
drock08 = ee.Image('users/ericamccormick/20_RockMoisture/deficit/bedrock/Dbedrock_2008')
drock09 = ee.Image('users/ericamccormick/20_RockMoisture/deficit/bedrock/Dbedrock_2009')
drock10 = ee.Image('users/ericamccormick/20_RockMoisture/deficit/bedrock/Dbedrock_2010')
drock11 = ee.Image('users/ericamccormick/20_RockMoisture/deficit/bedrock/Dbedrock_2011')
drock12 = ee.Image('users/ericamccormick/20_RockMoisture/deficit/bedrock/Dbedrock_2012')
drock13 = ee.Image('users/ericamccormick/20_RockMoisture/deficit/bedrock/Dbedrock_2013')
drock14 = ee.Image('users/ericamccormick/20_RockMoisture/deficit/bedrock/Dbedrock_2014')
drock15 = ee.Image('users/ericamccormick/20_RockMoisture/deficit/bedrock/Dbedrock_2015')
drock16 = ee.Image('users/ericamccormick/20_RockMoisture/deficit/bedrock/Dbedrock_2016')
drock17 = ee.Image('users/ericamccormick/20_RockMoisture/deficit/bedrock/Dbedrock_2017')

#Median Dbedrock,Y
Dbedrock_median_zeros = ee.Image('users/ericamccormick/20_RockMoisture/deficit/bedrock/Dbedrock_median')

#Masks
woodveg_x_shallowbedrock = ee.Image('users/ericamccormick/20_RockMoisture/masks/woodyVeg_x_shallowBedrock')
woodyveg = ee.Image('users/ericamccormick/20_RockMoisture/masks/mask_woodyVeg')
shallowbedrock = ee.Image('users/ericamccormick/20_RockMoisture/masks/mask_shallowBedrock')
mask_reproj = ee.Image('users/ericamccormick/20_RockMoisture/masks/mask_reproj') #final mask including ET>P

#Sbedrock and Sr
Sbedrock = ee.Image('users/ericamccormick/20_RockMoisture/deficit/bedrock/Sbedrock')
Sr = ee.Image('users/ericamccormick/20_RockMoisture/deficit/root-zone/Sr2003_2017_Mask')

#gNATSGO
Ssoil = ee.Image('users/ericamccormick/20_RockMoisture/products/gNATSGO/Ssoil_500m')
densic = ee.Image('users/ericamccormick/20_RockMoisture/products/gNATSGO/densic_weavg_lower_reprojected')
#lithic = 
paralithic = ee.Image('users/ericamccormick/20_RockMoisture/products/gNATSGO/paralithic_weavg_lower_reprojected')

#Other
koppen = ee.Image('users/ericamccormick/20_RockMoisture/products/Koppen_Global')
MAP = ee.Image('users/ericamccormick/MAP')
MeanAnnualPET = ee.Image('users/ericamccormick/MeanAnnualPET')
CA_abovegroundbiomass = ee.Image('users/ericamccormick/aboveground_biomass_carbon_2010_scaled_CA')


In [4]:
import pandas as pd
import numpy as np
from google.colab import files
import requests

##**Annual NPP**

In [ ]:
# already at 500m, scaling = 0.0001
npp = ee.ImageCollection("MODIS/006/MOD17A3HGF").select('Npp')

year = 2004
while (year < 2018):
  start = str(year) +"-01-01"
  stop = str(year)+"-01-31"
  to_export = npp.filterDate(start, stop).first().clip(conus.geometry()).toDouble()

  task_config = {
          'region': conus.geometry(),
          'fileFormat': 'GeoTIFF',
          'fileNamePrefix': 'NPP'+ str(year), #change the name to reflect parameters
          'image': to_export,
          'description': 'NPP'+str(year),
          'scale': 500,
          'maxPixels': 10000000000000,
          'crs':'EPSG:4326'
      }

  task=ee.batch.Export.image.toDrive(**task_config)
  task.start()

  year += 1

##**Mean NPP**

In [ ]:
npp = ee.ImageCollection("MODIS/006/MOD17A3HGF").select('Npp')
start = "2004-01-01"
stop = "2018-12-31"
to_export = npp.filterDate(start, stop).mean().clip(conus.geometry()).toDouble()

task_config = {
        'region': conus.geometry(),
        'fileFormat': 'GeoTIFF',
        'fileNamePrefix': 'NPP_mean', #change the name to reflect parameters
        'image': to_export,
        'description': 'NPP_mean',
        'scale': 500,
        'maxPixels': 10000000000000,
        'crs':'EPSG:4326'
    }

task=ee.batch.Export.image.toDrive(**task_config)
task.start()

##**Landfire**

In [ ]:
landfire = ee.Image('users/ericamccormick/20_RockMoisture/products/Landfire')

### This shouldn't be necessary b/c the pyramiding mode is set to 
landfire_500 = landfire.reduceResolution(**{
  'reducer': ee.Reducer.mode(),
  'maxPixels': 1024
  }).reproject(crs='EPSG:4326', scale = 500)


In [ ]:
task_config = {
        'region': conus.geometry(),
        'fileFormat': 'GeoTIFF',
        'fileNamePrefix': 'landfire_500_mode',
        'image': landfire_500.toDouble().clip(conus.geometry()),
        'description': 'landfire_500_mode',
        'scale': 500,
        'maxPixels': 10000000000000
    }

task=ee.batch.Export.image.toDrive(**task_config)
task.start()

##**Annual water year precipitation**

First calculate mean annual water year precipitation (MAP) from 2003-2017 using PRISM dataset (Daly, 2008).

In [ ]:
## define function to take the annual sum of precipitation 
def annualSum(startyear,stopyear):
  start = str(startyear) +"-10-01"
  stop = str(stopyear)+"-09-30"
  prism = ee.ImageCollection('OREGONSTATE/PRISM/AN81d').select('ppt').filterDate(start, stop)
  MAP = prism.reduce(**{
    'reducer': ee.Reducer.sum(),
  })
  return MAP

In [ ]:
## get the sum of precipitation each year
MAP2004 = annualSum(2003,2004)
MAP2005 = annualSum(2004,2005)
MAP2006 = annualSum(2005,2006)
MAP2007 = annualSum(2006,2007)
MAP2008 = annualSum(2007,2008)
MAP2009 = annualSum(2008,2009)
MAP2010 = annualSum(2009,2010)
MAP2011 = annualSum(2010,2011)
MAP2012 = annualSum(2011,2012)
MAP2013 = annualSum(2012,2013)
MAP2014 = annualSum(2013,2014)
MAP2015 = annualSum(2014,2015)
MAP2016 = annualSum(2015,2016)
MAP2017 = annualSum(2016,2017)

In [ ]:
all_years_p = [MAP2004, MAP2005, MAP2006, MAP2007, MAP2008, MAP2009, MAP2010, MAP2011, MAP2012, MAP2013, MAP2014, MAP2015, MAP2016, MAP2017]

In [ ]:
year = 2004
for i in all_years_p:
  to_export = i.clip(conus.geometry()).toDouble()

  task_config = {
          'region': conus.geometry(),
          'fileFormat': 'GeoTIFF',
          'fileNamePrefix': 'precip_'+ str(year), #change the name to reflect parameters
          'image': to_export,
          'description': str(year),
          'scale': 500,
          'maxPixels': 10000000000000,
          'crs':'EPSG:4326'
      }

  task=ee.batch.Export.image.toDrive(**task_config)
  task.start()

  year += 1

##**MAP**

In [ ]:
def annualSum(startyear,stopyear):
  start = str(startyear) +"-10-01"
  stop = str(stopyear)+"-09-30"
  prism = ee.ImageCollection('OREGONSTATE/PRISM/AN81d').select('ppt').filterDate(start, stop)
  MAP = prism.reduce(**{
    'reducer': ee.Reducer.sum(),
  })
  return MAP

In [ ]:
## get the sum of precipitation each year
MAP2004 = annualSum(2003,2004)
MAP2005 = annualSum(2004,2005)
MAP2006 = annualSum(2005,2006)
MAP2007 = annualSum(2006,2007)
MAP2008 = annualSum(2007,2008)
MAP2009 = annualSum(2008,2009)
MAP2010 = annualSum(2009,2010)
MAP2011 = annualSum(2010,2011)
MAP2012 = annualSum(2011,2012)
MAP2013 = annualSum(2012,2013)
MAP2014 = annualSum(2013,2014)
MAP2015 = annualSum(2014,2015)
MAP2016 = annualSum(2015,2016)
MAP2017 = annualSum(2016,2017)

In [ ]:
# Combine images from each year into an image collection with all years
MAP = ee.ImageCollection([MAP2004, MAP2005, MAP2006, MAP2007, MAP2008, MAP2009, MAP2010, MAP2011, MAP2012, MAP2013, MAP2014, MAP2015, MAP2016, MAP2017])

In [ ]:
# Get the mean of each cumulative annual precip
MAP_Mean = MAP.reduce(**{
    'reducer': ee.Reducer.mean(),
  })

In [ ]:
to_export = MAP_Mean.clip(conus.geometry()).toDouble()

task_config = {
        'region': conus.geometry(),
        'fileFormat': 'GeoTIFF',
        'fileNamePrefix': 'MAP', #change the name to reflect parameters
        'image': to_export,
        'description': 'MAP',
        'scale': 500,
        'maxPixels': 10000000000000,
        'crs':'EPSG:4326'
    }

task=ee.batch.Export.image.toDrive(**task_config)
task.start()

##**Annual spring precipitation**

In [ ]:
## define function to take the annual sum of precipitation 
def springSum(startyear,stopyear):
  start = str(startyear) +"-3-01"
  stop = str(stopyear)+"-05-31"
  prism = ee.ImageCollection('OREGONSTATE/PRISM/AN81d').select('ppt').filterDate(start, stop)
  MAP = prism.reduce(**{
    'reducer': ee.Reducer.sum(),
  })
  return MAP

In [ ]:
## get the sum of precipitation each year
MAP2004 = annualSum(2003,2004)
MAP2005 = annualSum(2004,2005)
MAP2006 = annualSum(2005,2006)
MAP2007 = annualSum(2006,2007)
MAP2008 = annualSum(2007,2008)
MAP2009 = annualSum(2008,2009)
MAP2010 = annualSum(2009,2010)
MAP2011 = annualSum(2010,2011)
MAP2012 = annualSum(2011,2012)
MAP2013 = annualSum(2012,2013)
MAP2014 = annualSum(2013,2014)
MAP2015 = annualSum(2014,2015)
MAP2016 = annualSum(2015,2016)
MAP2017 = annualSum(2016,2017)

In [ ]:
all_years_spring_p = [MAP2004, MAP2005, MAP2006, MAP2007, MAP2008, MAP2009, MAP2010, MAP2011, MAP2012, MAP2013, MAP2014, MAP2015, MAP2016, MAP2017]

In [ ]:
year = 2004
for i in all_years_spring_p:
  to_export = i.clip(conus.geometry()).toDouble()

  task_config = {
          'region': conus.geometry(),
          'fileFormat': 'GeoTIFF',
          'fileNamePrefix': 'spring_precip_'+ str(year), #change the name to reflect parameters
          'image': to_export,
          'description': 'spring' + str(year),
          'scale': 500,
          'maxPixels': 10000000000000,
          'crs':'EPSG:4326'
      }

  task=ee.batch.Export.image.toDrive(**task_config)
  task.start()

  year += 1

##**Mean annual ET (PML)**

In [ ]:
def annualSum_ET(startyear,stopyear):
  start = str(startyear) +"-10-01"
  stop = str(stopyear)+"-09-30"
  es = ee.ImageCollection('CAS/IGSNRR/PML/V2').select('Es').filterDate(start, stop)
  ec = ee.ImageCollection('CAS/IGSNRR/PML/V2').select('Ec').filterDate(start, stop)
  Es = es.reduce(**{
    'reducer': ee.Reducer.sum(),
  })
  Ec = ec.reduce(**{
    'reducer': ee.Reducer.sum(),
  })

  ET = Es.add(Ec)
  ET = ET.multiply(8) # because its 8 day resolution

  return ET

In [ ]:
## get the sum of precipitation each year
MAP2004 = annualSum_ET(2003,2004)
MAP2005 = annualSum_ET(2004,2005)
MAP2006 = annualSum_ET(2005,2006)
MAP2007 = annualSum_ET(2006,2007)
MAP2008 = annualSum_ET(2007,2008)
MAP2009 = annualSum_ET(2008,2009)
MAP2010 = annualSum_ET(2009,2010)
MAP2011 = annualSum_ET(2010,2011)
MAP2012 = annualSum_ET(2011,2012)
MAP2013 = annualSum_ET(2012,2013)
MAP2014 = annualSum_ET(2013,2014)
MAP2015 = annualSum_ET(2014,2015)
MAP2016 = annualSum_ET(2015,2016)
MAP2017 = annualSum_ET(2016,2017)

In [ ]:
# Combine images from each year into an image collection with all years
MAP = ee.ImageCollection([MAP2004, MAP2005, MAP2006, MAP2007, MAP2008, MAP2009, MAP2010, MAP2011, MAP2012, MAP2013, MAP2014, MAP2015, MAP2016, MAP2017])

In [ ]:
# Get the mean of each cumulative annual precip
MAP_Mean = MAP.reduce(**{
    'reducer': ee.Reducer.mean(),
  })

In [ ]:
task_config = {
        'region': conus.geometry(),
        'fileFormat': 'GeoTIFF',
        'fileNamePrefix': 'mean_annual_et_pml_wx8', #change the name to reflect parameters
        'image': MAP_Mean.clip(conus.geometry()).toDouble(),
        'description': 'mean_annual_et_pml_wx8',
        'scale': 500,
        'maxPixels': 10000000000000,
        'crs':'EPSG:4326'
    }

task=ee.batch.Export.image.toDrive(**task_config)
task.start()

##**Summer ET (PML)** (June, July, August)

In [11]:
def summer_ET(startyear,stopyear):
  start = str(startyear) +"-6-01"
  stop = str(stopyear)+"-8-31"
  es = ee.ImageCollection('CAS/IGSNRR/PML/V2').select('Es').filterDate(start, stop)
  ec = ee.ImageCollection('CAS/IGSNRR/PML/V2').select('Ec').filterDate(start, stop)
  Es = es.reduce(**{
    'reducer': ee.Reducer.sum(),
  })
  Ec = ec.reduce(**{
    'reducer': ee.Reducer.sum(),
  })

  ET = Es.add(Ec)
  ET = ET.multiply(8) # because its 8 day resolution

  return ET

In [12]:
## get the sum of precipitation each year
MAP2004 = summer_ET(2003,2004)
MAP2005 = summer_ET(2004,2005)
MAP2006 = summer_ET(2005,2006)
MAP2007 = summer_ET(2006,2007)
MAP2008 = summer_ET(2007,2008)
MAP2009 = summer_ET(2008,2009)
MAP2010 = summer_ET(2009,2010)
MAP2011 = summer_ET(2010,2011)
MAP2012 = summer_ET(2011,2012)
MAP2013 = summer_ET(2012,2013)
MAP2014 = summer_ET(2013,2014)
MAP2015 = summer_ET(2014,2015)
MAP2016 = summer_ET(2015,2016)
MAP2017 = summer_ET(2016,2017)

In [13]:
all_years_p = [MAP2004, MAP2005, MAP2006, MAP2007, MAP2008, MAP2009, MAP2010, MAP2011, MAP2012, MAP2013, MAP2014, MAP2015, MAP2016, MAP2017]

In [14]:
year = 2004
for i in all_years_p:
  to_export = i.clip(conus.geometry()).toDouble()

  task_config = {
          'region': conus.geometry(),
          'fileFormat': 'GeoTIFF',
          'fileNamePrefix': 'ET_summer_'+ str(year), #change the name to reflect parameters
          'image': to_export,
          'description': 'ET_summer_'+str(year),
          'scale': 500,
          'maxPixels': 10000000000000,
          'crs':'EPSG:4326'
      }

  task=ee.batch.Export.image.toDrive(**task_config)
  task.start()

  year += 1

##**EARLY Summer ET (PML)** (April, May)

In [18]:
def EARLYsummer_ET(startyear,stopyear):
  start = str(startyear) +"-4-01"
  stop = str(stopyear)+"-5-30"
  es = ee.ImageCollection('CAS/IGSNRR/PML/V2').select('Es').filterDate(start, stop)
  ec = ee.ImageCollection('CAS/IGSNRR/PML/V2').select('Ec').filterDate(start, stop)
  Es = es.reduce(**{
    'reducer': ee.Reducer.sum(),
  })
  Ec = ec.reduce(**{
    'reducer': ee.Reducer.sum(),
  })

  ET = Es.add(Ec)
  ET = ET.multiply(8) # because its 8 day resolution

  return ET

In [19]:
## get the sum of precipitation each year
MAP2004 = EARLYsummer_ET(2003,2004)
MAP2005 = EARLYsummer_ET(2004,2005)
MAP2006 = EARLYsummer_ET(2005,2006)
MAP2007 = EARLYsummer_ET(2006,2007)
MAP2008 = EARLYsummer_ET(2007,2008)
MAP2009 = EARLYsummer_ET(2008,2009)
MAP2010 = EARLYsummer_ET(2009,2010)
MAP2011 = EARLYsummer_ET(2010,2011)
MAP2012 = EARLYsummer_ET(2011,2012)
MAP2013 = EARLYsummer_ET(2012,2013)
MAP2014 = EARLYsummer_ET(2013,2014)
MAP2015 = EARLYsummer_ET(2014,2015)
MAP2016 = EARLYsummer_ET(2015,2016)
MAP2017 = EARLYsummer_ET(2016,2017)

In [20]:
all_years_p = [MAP2004, MAP2005, MAP2006, MAP2007, MAP2008, MAP2009, MAP2010, MAP2011, MAP2012, MAP2013, MAP2014, MAP2015, MAP2016, MAP2017]

In [21]:
year = 2004
for i in all_years_p:
  to_export = i.clip(conus.geometry()).toDouble()

  task_config = {
          'region': conus.geometry(),
          'fileFormat': 'GeoTIFF',
          'fileNamePrefix': 'ET_AprMay_'+ str(year), #change the name to reflect parameters
          'image': to_export,
          'description': 'ET_AprMay_'+str(year),
          'scale': 500,
          'maxPixels': 10000000000000,
          'crs':'EPSG:4326'
      }

  task=ee.batch.Export.image.toDrive(**task_config)
  task.start()

  year += 1

##**Annual PET (MODIS)**

In [ ]:
def annualSum_PET(startyear,stopyear):
  start = str(startyear) +"-10-01"
  stop = str(stopyear)+"-09-30"
  modis_pet = ee.ImageCollection("MODIS/006/MOD16A2").select('PET').filterDate(start, stop)

  pet = modis_pet.reduce(**{
    'reducer': ee.Reducer.sum(),
  })

  pet = pet.divide(10)

  return pet

In [ ]:
MAP2004 = annualSum_PET(2003,2004)
MAP2005 = annualSum_PET(2004,2005)
MAP2006 = annualSum_PET(2005,2006)
MAP2007 = annualSum_PET(2006,2007)
MAP2008 = annualSum_PET(2007,2008)
MAP2009 = annualSum_PET(2008,2009)
MAP2010 = annualSum_PET(2009,2010)
MAP2011 = annualSum_PET(2010,2011)
MAP2012 = annualSum_PET(2011,2012)
MAP2013 = annualSum_PET(2012,2013)
MAP2014 = annualSum_PET(2013,2014)
MAP2015 = annualSum_PET(2014,2015)
MAP2016 = annualSum_PET(2015,2016)
MAP2017 = annualSum_PET(2016,2017)



In [ ]:
all_years_p = [MAP2004, MAP2005, MAP2006, MAP2007, MAP2008, MAP2009, MAP2010, MAP2011, MAP2012, MAP2013, MAP2014, MAP2015, MAP2016, MAP2017]

In [ ]:
year = 2004
for i in all_years_p:
  to_export = i.clip(conus.geometry()).toDouble()

  task_config = {
          'region': conus.geometry(),
          'fileFormat': 'GeoTIFF',
          'fileNamePrefix': 'PET'+ str(year), #change the name to reflect parameters
          'image': to_export,
          'description': str(year),
          'scale': 500,
          'maxPixels': 10000000000000,
          'crs':'EPSG:4326'
      }

  task=ee.batch.Export.image.toDrive(**task_config)
  task.start()

  year += 1

##**Mean annual PET (MODIS)**

In [ ]:
def annualSum_PET(startyear,stopyear):
  start = str(startyear) +"-10-01"
  stop = str(stopyear)+"-09-30"
  modis_pet = ee.ImageCollection("MODIS/006/MOD16A2").select('PET').filterDate(start, stop)

  pet = modis_pet.reduce(**{
    'reducer': ee.Reducer.sum(),
  })

  pet = pet.divide(10)

  return pet

In [ ]:
MAP2004 = annualSum_PET(2003,2004)
MAP2005 = annualSum_PET(2004,2005)
MAP2006 = annualSum_PET(2005,2006)
MAP2007 = annualSum_PET(2006,2007)
MAP2008 = annualSum_PET(2007,2008)
MAP2009 = annualSum_PET(2008,2009)
MAP2010 = annualSum_PET(2009,2010)
MAP2011 = annualSum_PET(2010,2011)
MAP2012 = annualSum_PET(2011,2012)
MAP2013 = annualSum_PET(2012,2013)
MAP2014 = annualSum_PET(2013,2014)
MAP2015 = annualSum_PET(2014,2015)
MAP2016 = annualSum_PET(2015,2016)
MAP2017 = annualSum_PET(2016,2017)



In [ ]:
# Combine images from each year into an image collection with all years
MAP = ee.ImageCollection([MAP2004, MAP2005, MAP2006, MAP2007, MAP2008, MAP2009, MAP2010, MAP2011, MAP2012, MAP2013, MAP2014, MAP2015, MAP2016, MAP2017])

In [ ]:
# Get the mean of each cumulative annual precip
MAP_Mean = MAP.reduce(**{
    'reducer': ee.Reducer.mean(),
  })

In [ ]:
task_config = {
        'region': conus.geometry(),
        'fileFormat': 'GeoTIFF',
        'fileNamePrefix': 'mean_annual_pet_modis', #change the name to reflect parameters
        'image': MAP_Mean.clip(conus.geometry()).toDouble(),
        'description': 'mean_annual_pet_modis',
        'scale': 500,
        'maxPixels': 10000000000000,
        'crs':'EPSG:4326'
    }

task=ee.batch.Export.image.toDrive(**task_config)
task.start()

##**Biomass**

In [ ]:
# Note this is from erica.elmstead's gee account
biomass = ee.Image('users/ericaelmstead/Aboveground_Biomass_US')

task_config = {
        'region': conus.geometry(),
        'fileFormat': 'GeoTIFF',
        'fileNamePrefix': 'biomass', #change the name to reflect parameters
        'image': biomass.clip(conus.geometry()).toDouble(),
        'description': 'biomass',
        'scale': 500,
        'maxPixels': 10000000000000,
        'crs':'EPSG:4326'
    }

task=ee.batch.Export.image.toDrive(**task_config)
task.start()

##**Avg # of days of snow cover >10% and mean annual PET available already from previous analysis**